In [4]:
import pandas as pd
import numpy as np
from DataReader import split_sheet_into_test_training_per_word
from nltk.tokenize import regexp_tokenize
from nltk import bigrams
from nltk import FreqDist
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
x_test, x_train, y_test, y_train = split_sheet_into_test_training_per_word()

In [30]:
n=2
start_symbol = '<s>'
x_train_clean = []
for index, row in x_train.iterrows():
    if row['probe'] in row['line']:
        line = regexp_tokenize(row['line'], "[\w']+")
        label = row['probe']+str(y_train.loc[index][0])
        ind= line.index(row['probe'])
        if ind-n+1>0:
            s= line[ind-n+1:ind]
            s.append(label)
            x_train_clean.append(s)
        elif ind!=0:
            s = [start_symbol]*(n-1-ind)
            s.extend(line[0:ind])
            s.append(label)
            x_train_clean.append(s)
        else:
            s = [start_symbol]*(n-1)
            s.append(label)
            x_train_clean.append(s)

In [31]:
train_data, padded_sents = padded_everygram_pipeline(n, x_train_clean)

model= MLE(n)
model.fit(train_data, padded_sents)

In [32]:
correct, error, noidea=0,0,0

for index, row in x_test.iterrows():
    p=[]
    for i in range(4):
        line= regexp_tokenize(row['line'], "[\w']+")
        ind = line.index(row['probe'])
        prior =[]
        if ind-n+1>0:
            s= line[ind-n+1:ind]
            prior.extend(s)
        elif ind!=0:
            s = [start_symbol]*(n-1-ind)
            s.extend(line[0:ind])
            prior.extend(s)
        else:
            s = [start_symbol]*(n-1)
            prior.extend(s)
        p.append(model.score(row['probe']+str(i+1),prior))
    if sum(p)<0.00001:
        noidea+=1
    elif p.index(max(p))+1 == y_test.loc[index][0]:
        correct+=1
    else:
        error+=1

In [33]:
print(correct)
print(error)
print(noidea)

7403
268
3714
